In [5]:
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [6]:
# Model imports
from apps.activity.models import Asset, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt, Contract, ContractDetail, GeofenceMaster, Shift, SitePeople, TypeAssist, WizardDraft
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery

In [3]:
# Apps utils imports
from apps.y_helpdesk.utils import send_ticket_email
from apps.core import utils


In [19]:
Pgbelonging.objects.get_assigned_sites_to_people(52).values_list('bucode')


<QuerySet [('ICICI ASHER KOLSHET Rd', 'MUM0020', 10), ('ICICI BHIWANDI', 'MUM00021', 28), ('ICICI Indiranagar', 'BAN0001', 23), ('ICICI Phoenix Market City', 'MUM00_3', 39), ('ICICI Ridge Street', 'MUM027', 31)]>

In [21]:
p = People.objects.get(id=52)

In [23]:
p.isverified

False

In [6]:
QuestionSet.objects.filter(type = QuestionSet.Type.CHECKLIST)

<QuerySet [<QuestionSet: Youtility AC checklist>, <QuestionSet: testChecklist>, <QuestionSet: Equipment Checklist>, <QuestionSet: Restroom Checklist>, <QuestionSet: Room Checklist>, <QuestionSet: testCheckListUnified>]>

In [28]:
caps = Capability.objects.select_related('parent').filter(~Q(capscode='NONE'),  cfor = 'WEB').order_by('-parent')

In [45]:
web_caps = []
current_parent = ""
for i in caps:
    childs = []
    if i.parent.capscode == 'NONE':
        web_caps.append(i.capscode)
        current_parent = i.capscode
    if i.parent.capscode == current_parent:
        childs.append([i.capscode, i.capsname])
    else:
        web_caps.append(childs)

In [46]:
web_caps

[[],
 [],
 'DASHBOARD',
 [],
 'TRACKING',
 [],
 'ONBOARDING',
 [],
 'SCHEDULING',
 [],
 'HELPDESK',
 [],
 'ADMIN',
 [],
 [],
 [],
 [],
 'COMPANYSETUP',
 []]

In [11]:
from apps.core.raw_queries import get_query
web = Capability.objects.raw(get_query('get_web_caps_for_client'))

In [12]:
for ele in web:
    if ele.cfor == 'WEB': 
        print(ele.id, ele.capscode, ele.parent_id, ele.cfor)

1 NONE None WEB
101 ADMIN 1 WEB
102 TYPEASSIST 101 WEB
103 REPORTS 101 WEB
187 SITESURVEY 1 WEB
105 SRY_SITEREPORT 187 WEB
106 SRY_CONVEYANCE 187 WEB
72 DASHBOARD 1 WEB
73 TRACKING 1 WEB
74 GEOFENCETRACKING 73 WEB
75 SITETOURTRACKING 73 WEB
76 MOBILEUSERLOG 73 WEB
77 MOBILEUSERDETAILS 73 WEB
78 SOS 73 WEB
79 SSINCIDENTREPORT 73 WEB
80 PEOPLENEARASSET 73 WEB
81 CCTVEVENTS 73 WEB
82 ONBOARDING 1 WEB
181 COMPANYSETUP 82 WEB
83 CS_BUSINESSVIEW 181 WEB
84 CS_PEOPLE 181 WEB
85 CS_SITEGROUP 181 WEB
86 SHIFT 82 WEB
87 PEOPLEGROUP 82 WEB
88 LOCATION 82 WEB
89 ASSET 82 WEB
90 CHECKPOINT 82 WEB
91 QUESTION 82 WEB
92 QUESTIONSET 82 WEB
93 SCHEDULING 1 WEB
94 TASK 93 WEB
95 PPMPLAN 93 WEB
96 TOUR 93 WEB
97 ROUTEPLAN 93 WEB
98 HELPDESK 1 WEB
100 TICKETESCALATION 98 WEB
99 TICKETS 98 WEB


In [6]:
Capability.objects.filter(enable=False).delete()

(1, {'peoples.Capability': 1})